# The IMDB Dataset

---

### Colab Note

Don't forget that you can link your notebook to your drive and save your work there. Then you can download and backup your models, reload them to keep training them, or upload datasets to your drive.

```python
import os
import sys

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('drive/My Drive/') # 'My Drive' is the default name of Google Drives
    os.listdir()

# use os.chdir("my-directory") # to change directory, and
# os.listdir()                 # to list its contents
# os.getcwd()                  # to get the name of the current directory
# os.mkdir("my-new-dir")       # to create a new directory
# See: https://realpython.com/working-with-files-in-python/

# You can also use bash commands directly, preceded by a bang
# !ls
# However, the following will *not* change the Python directory
# the notebook points to (use os.chdir for that)!
# !cd my-directory
```

---

## 1. Theory

Make sure you understand the first video of 3Blue1Brown's introduction to neural nets, and ask questions if there's anything unclear.

[3Blue1Brown | Backpropagation, intuitively | Deep Learning Chapter 3](https://www.youtube.com/watch?v=Ilg3gGewQ5U)

The fourth video is optional.

[3Blue1Brown | Backpropagation calculus | Deep Learning Chapter 4 ](https://www.youtube.com/watch?v=tIeHLnjs5U8)

---

## 2. Practice

In [4]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras

### For reproducible results

In Keras ([source](https://keras.io/examples/keras_recipes/reproducibility_recipes/)):
```python
keras.utils.set_random_seed(812) # See below

# If using TensorFlow, this will make GPU ops as deterministic as possible,
# but it will affect the overall performance, so be mindful of that.
tf.config.experimental.enable_op_determinism()
```

Note: `keras.utils.set_random_seed` will do the following ([source](https://github.com/keras-team/keras/blob/d66ecf029e4864eeeff7e22408e82c95d63422d0/keras/src/utils/rng_utils.py#L58)):

```python
import random
random.seed(42)

import numpy as np
np.random.seed(42)

# if tf as backend
tf.random.set_seed(42)

# if torch as backend
torch.manual_seed(42)
```

### IMDB Set-up

In [5]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.0
    return results


# load the IMDB dataset (max review length
(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=10000)

# preprocess
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

# split training set into train & validation
partial_x_train = x_train[10000:]
partial_y_train = y_train[10000:]
x_val = x_train[:10000]
y_val = y_train[:10000]

In [ ]:
# build
model = keras.models.Sequential()
model.add(keras.Input((10000,)))
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

Sanity check, how does our network perform before training (use `.evaluate` on `partial_x_train, partial_y_train`). Is the accuracy a value you would expect?

Now we can train.

In [ ]:
# save data from training into the 'history' object
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=128,
    validation_data=(x_val, y_val),
)

### Visualise your results

Thanks to these plots, it is easier to spot the epoch (epoch number) where our net reached peak performance (lowest *validation loss*/highest *validation accuracy*, prioritising accuracy if the two are not the same).

Think about what would be a good strategy to keep your code as organised as possible as you run many experiments?

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
history_dict["loss"]

In [ ]:
loss = history_dict["loss"]
val_loss = history_dict["val_loss"]

epochs = range(1, len(loss) + 1)

blue_dots = "bo"
solid_blue_line = "b"

plt.plot(epochs, loss, blue_dots, label="Training loss")
plt.plot(epochs, val_loss, solid_blue_line, label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.show()

In [ ]:
history_dict["accuracy"]

In [ ]:
plt.clf()

acc = history_dict["accuracy"]
val_acc = history_dict["val_accuracy"]

epochs = range(1, len(acc) + 1)

blue_dots = "bo"
solid_blue_line = "b"

plt.plot(epochs, acc, blue_dots, label="Training acc")
plt.plot(epochs, val_acc, solid_blue_line, label="Validation acc")
plt.title("Training and validation acc")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.show()

Can you think of a way to making the plotting code above more re-usable, when training multiple models?

### Experiments

- Experiment with only one layer, then with more (deeper net);
- Experiment with more or fewer hidden units – 32 units, 64 units etc.
  - One nice challenge is to see how good your results get with a bigger network, then see if you can get to the same level with a smaller one by training longer, or tweaking the learning rate/changing the optimizer;
  - Another is to see how good a result you can get with a fixed number of epochs (e.g. 5);
- Experiment with replacing `relu` with `tanh` activations;
- Try the [`Adam` optimizer](https://keras.io/api/optimizers/adam/): `optimizers.Adam(learning_rate=0.001)`
- Investigate the effect of different learning rates;
- Investigate the effect of a smaller (or bigger) batch size;
- Train for more epochs, or, conversely, set a number of epochs (say 5), and see how good you can get your model in just these 5 epochs.

**Think about how to record and organise your experiments in a neat way!**

## 3. Conclusion

Take your best network and train on **all the training data** (`x_train`, `y_train`), without a train/validation split, using the same hyperparameters (optimizer, learning rate, network size, etc.) as your best run, for the optimal number of epochs (looking at your best validation curves).

Evaluate this last model on the test set (`x_test, y_test`).

### Use your model (optional)

Can you import the lecture code used to test the model on a review, and see if you agree with its prediction?

### Save and load models

To save and load models locally, you can use [the high-level API](https://www.tensorflow.org/tutorials/keras/save_and_load):
```python
model.save("my_imdb_model.keras")
```
Later one, to reload it, use:
```python
reloaded_model = keras.models.load_model('my_imdb_model.keras')
```

It is also possible to save not just the model, but also the state of your optimiser, and every variable used during training, using the morer involved [checkpoints](https://www.tensorflow.org/guide/checkpoint#create_the_checkpoint_objects).

## 4. Additional experiments

- One parameter you could study is the influence on the vocabulary size on the final results: you might then want to store the vocab size in a variable, and use that instead of the hard-coded `10000` that we have.
- It would be entirely possible to approach the same problem as if you were detecting two classes (`0` for negative, `1` for positive), instead of using just a single output. In order to implement that, you would need:
   1. two units in your final layer with a `softmax` activation;
   2. `categorical_crossentropy` as a loss;
   3. labels as one-hot vectors (using `keras.utils.to_categorical`);
   4. and `np.argmax` (instead of `round`) to retrieve the likeliest class (the index) when making predictions.
- Another line of enquiry is the study of the behaviour of your trained model:
  - Are you able to modify existing reviews in a way that changes the initial prediction of your model? (One 'automated' way of doing that would be to remove a certain number of words from the review, and see how performance is impacted by that information loss.)
  - Are you able to create a pipeline where you write your own review, or find one online, transform it into the appropriate format (remove punctuation, turn everything to lower case, convert to an array of integers using the dictionary yielded by `keras.datasets.imdb.get_word_index()` (beware of the shift by 3 induced by the reserved tokens for padding, start of sequence and unknown!), and see what prediction you get for it?